# N starvation 

## Which N sources allow MED4 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [6]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [7]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return pd.Series(res)

# Import model and manipulate based on Ofaim at el

In [8]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [9]:
model = cobra.io.read_sbml_model(model_fpath)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


# Get a DF of all MED4 metabolites

In [10]:
df_m = pd.DataFrame.from_records([
    {
        'mid': m.id,
        'name' : m.name,
        'formula' : m.formula,
        'charge' : m.charge,
    }
    for m in model.metabolites
])

df_m.head()

,mid,name,formula,charge
0,amino_acid_pool[c],Amino Acid Pool biomass precursor,,0
1,Amino_sugars[c],Amino sugars pool,,0
2,Artificial_protein[c],Artificial Protein biomass precursor,,0
3,biomass[c],Biomass,,0
4,H2O[c],H2O,H2O,0


In [11]:
df_m.formula.value_counts()

                165
C40H56            7
H                 5
C5H9NO4           4
C40H58            4
               ... 
C14H23N6O3S       1
C11H23N2O7PS      1
C8H13NO6          1
C10H13N2O11P      1
C6H9N2O5P         1
Name: formula, Length: 470, dtype: int64

# Load compounds DB from modelseed

In [12]:
fpath = '~/work/tools//ModelSEEDDatabase/Biochemistry/compounds.tsv'
ref_df = pd.read_csv(fpath, sep='\t')

/tmp/ipykernel_2325877/2212162129.py:2: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_df = pd.read_csv(fpath, sep='\t')


In [13]:
ref_df['BiggID'] = ref_df.aliases.str.extract(r'BiGG: ([^;\|\:]+)')

In [14]:
df_m['compartment'] = df_m.mid.str.extract(r'(\[[^\]]+\])$')

In [15]:
# match MED4 to the ref DF

In [16]:
ref_df.head()

,id,abbreviation,name,formula,mass,source,inchikey,charge,is_core,is_obsolete,...,deltag,deltagerr,pka,pkb,abstract_compound,comprised_of,aliases,smiles,notes,BiggID
0,cpd00001,h2o,H2O,H2O,18.0,Primary Database,XLYOFNOQVPJJNP-UHFFFAOYSA-N,0,1,0,...,-37.54,0.18,1:1:15.70,1:1:-1.80,NaN,NaN,Name: H20; H2O; H3O+; HO-; Hydroxide ion; OH; ...,O,GC|EQ|EQU,h2o
1,cpd00002,atp,ATP,C10H13N5O13P3,504.0,Primary Database,ZKHQWZAMYRWXGA-KQYNXXCUSA-K,-3,1,0,...,-548.85,0.36,1:14:12.60;1:22:3.29;1:26:0.90;1:29:7.42;1:30:...,1:6:-7.46;1:9:-1.06;1:14:-3.85;1:15:4.93,NaN,NaN,Name: ATP; Adenosine 5'-triphosphate; adenosin...,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O...,GC|EQ|EQU,atp
2,cpd00003,nad,NAD,C21H26N7O14P2,662.0,Primary Database,BAWFJGJZGIEFAR-NNYOXOHSSA-M,-1,1,0,...,-286.41,1.59,1:6:11.94;1:17:1.85;1:18:2.28;1:25:11.38;1:35:...,1:6:-4.22;1:35:-3.85;1:37:-1.05;1:41:4.93;1:43...,NaN,NaN,Name: DPN; DPN+; DPN-ox; Diphosphopyridine nuc...,NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)([O-])OP(...,GC|EQ|EQU,nad
3,cpd00004,nadh,NADH,C21H27N7O14P2,663.0,Primary Database,BOPGDPNILDQYTO-NNYOXOHSSA-L,-2,1,0,...,-271.15,1.59,1:14:12.28;1:18:14.00;1:22:-7.46;1:26:-1.05;1:...,1:6:2.28;1:9:1.85;1:14:-3.85;1:15:4.93;1:18:-3...,NaN,NaN,Name: DPNH; NAD-reduced; NADH; NADH+H+; NADH2;...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...,GC|EQ|EQU,nadh
4,cpd00005,nadph,NADPH,C21H26N7O17P3,742.0,Primary Database,ACFIXJIJDZMPPO-NNYOXOHSSA-J,-4,1,0,...,-483.10,1.62,1:18:0.90;1:19:5.78;1:26:0.66;1:30:3.26;1:40:1...,1:11:-7.46;1:12:-1.06;1:22:4.87;1:40:-3.78,NaN,NaN,Name: NADP(H); NADP-red; NADP-reduced; NADPH; ...,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...,GC|EQ|EQU,nadph


In [17]:
ref_df = ref_df.loc[ref_df.source.isin(['Primary Database'])]


In [18]:
# based on Shani's refill code

#s=s.replace('n ','')
replacement_list = [
    (',', '_'), 
    ('(',''),
    (')',''),
    ("'",''),
    (' ','_'),
    ('-','_'),    
    ('[c]', ''),
    ('[e]', ''),
    ('[p]', ''),
    ('[th]', ''),
    ('[car]', ''),
]
ref_df['cleaned_name'] = ref_df['name']

for i,j in replacement_list:
    ref_df['cleaned_name'] = ref_df['cleaned_name'].str.replace(i,j,regex=False)
ref_df['cleaned_name'] = ref_df['cleaned_name'].str.lower()


In [19]:
ref_df['alias'] = ref_df.aliases.str.split(r';|\||: ')
eref_df = ref_df.explode('alias')
eref_df['cleaned_alias'] = eref_df['alias'].str.strip()

for i,j in replacement_list:
    eref_df['cleaned_alias'] = eref_df['cleaned_alias'].str.replace(i,j,regex=False)

eref_df['cleaned_alias'] = eref_df['cleaned_alias'].str.strip().str.lower()

In [20]:
df_m['cleaned_name'] = df_m['name']

for i,j in replacement_list:
    df_m['cleaned_name'] = df_m['cleaned_name'].str.replace(i,j,regex=False)

df_m['cleaned_mid'] = df_m['mid']

for i,j in replacement_list:
    df_m['cleaned_mid'] = df_m['cleaned_mid'].str.replace(i,j,regex=False)

    
# replace names

med4_replace_names = {
    'S_Malate' : 'L_Malate',
    'Divinylchlorophyll_b' : 'bacterio_chlorophyll_b',
    'Cadmium' : 'cd2+',
    'Copper' : 'cui'
#     '(R)_2,3_Dihydroxy_3_methylbutanoate' : 'R_2_3_Dihydroxy_3_methylbutanoate',
#     'trans_Dodec_2_enoyl_[acp]' :'trans_Dodec_2_enoyl',
#     "9,15,9'_tricis_zeta_Carotene" :'9_15_9_tricis_zeta_Carotene',
#     'Deamino_NAD': 'Deamino',
#     "5'_Phosphoribosyl_N_formylglycinamide":'5_Phosphoribosyl_N_formylglycinamide',
#     "5'_Phosphoribosylglycinamide" :'5_Phosphoribosylglycinamide',
#     "D_4'_Phosphopantothenate" :'D_4_Phosphopantothenate',
#     "7,8_Dihydroneopterin_3'_triphosphate" :'7_8_Dihydroneopterin_3_triphosphate',
    
}
df_m['cleaned_name'] = df_m['cleaned_name'].replace(med4_replace_names)
df_m['cleaned_name'] = df_m['cleaned_name'].str.lower()
df_m['cleaned_mid'] = df_m['cleaned_mid'].replace(med4_replace_names)
df_m['cleaned_mid'] = df_m['cleaned_mid'].str.lower()

df_m.head()

,mid,name,formula,charge,compartment,cleaned_name,cleaned_mid
0,amino_acid_pool[c],Amino Acid Pool biomass precursor,,0,[c],amino_acid_pool_biomass_precursor,amino_acid_pool
1,Amino_sugars[c],Amino sugars pool,,0,[c],amino_sugars_pool,amino_sugars
2,Artificial_protein[c],Artificial Protein biomass precursor,,0,[c],artificial_protein_biomass_precursor,artificial_protein
3,biomass[c],Biomass,,0,[c],biomass,biomass
4,H2O[c],H2O,H2O,0,[c],h2o,h2o


In [21]:
mdf = pd.merge(df_m, ref_df, on='cleaned_name', suffixes=['','_ref'], how='left')

In [22]:
mdf.name_ref.isna()

0       True
1       True
2       True
3       True
4      False
       ...  
805     True
806    False
807     True
808    False
809     True
Name: name_ref, Length: 810, dtype: bool

In [23]:
unmapped_ids = mdf.loc[mdf.name_ref.isna(), 'mid']
mdf2 = pd.merge(df_m.loc[df_m.mid.isin(unmapped_ids)
                        ], ref_df,left_on='cleaned_mid', right_on='cleaned_name', suffixes=['','_ref'], how='left')

In [24]:
unmapped_ids = mdf2.loc[mdf2.name_ref.isna(), 'mid']
mdf3 = pd.merge(df_m.loc[df_m.mid.isin(unmapped_ids)
                        ], eref_df,left_on='cleaned_name', right_on='cleaned_alias', suffixes=['','_ref'], how='left')

In [25]:
unmapped_ids = mdf3.loc[mdf3.name_ref.isna(), 'mid']
mdf4 = pd.merge(df_m.loc[df_m.mid.isin(unmapped_ids)
                        ], eref_df,left_on='cleaned_mid', right_on='cleaned_alias', suffixes=['','_ref'], how='left')

In [26]:
unmapped_ids = mdf4.loc[mdf4.name_ref.isna(), 'mid']
mdf5 = pd.merge(df_m.loc[df_m.mid.isin(unmapped_ids)
                        ], ref_df,left_on='formula', right_on='formula', suffixes=['','_ref'], how='left')

In [27]:
mdf5.loc[mdf5['id'].isna()]

,mid,name,formula,charge,compartment,cleaned_name,cleaned_mid,id,abbreviation,name_ref,...,pka,pkb,abstract_compound,comprised_of,aliases,smiles,notes,BiggID,cleaned_name_ref,alias
0,amino_acid_pool[c],Amino Acid Pool biomass precursor,,0,[c],amino_acid_pool_biomass_precursor,amino_acid_pool,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amino_sugars[c],Amino sugars pool,,0,[c],amino_sugars_pool,amino_sugars,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Artificial_protein[c],Artificial Protein biomass precursor,,0,[c],artificial_protein_biomass_precursor,artificial_protein,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,biomass[c],Biomass,,0,[c],biomass,biomass,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Photon_P680[c],Photon absorbed by PSII,,0,[c],photon_absorbed_by_psii,photon_p680,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,[Protein]_N6_octanoyl_L_lysine[c],[Protein]_N6_octanoyl_L_lysine[c],,0,[c],[protein]_n6_octanoyl_l_lysine,[protein]_n6_octanoyl_l_lysine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Thiocarboxy_[sulfur_carrier_protein][c],Thiocarboxy_[sulfur_carrier_protein][c],,0,[c],thiocarboxy_[sulfur_carrier_protein],thiocarboxy_[sulfur_carrier_protein],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,UDPMurNAcoyl_L_Ala_D_gamma_Glu_L_Lys_D_Ala_D_A...,UDPMurNAcoyl_L_Ala_D_gamma_Glu_L_Lys_D_Ala_D_A...,,0,[c],udpmurnacoyl_l_ala_d_gamma_glu_l_lys_d_ala_d_ala,udpmurnacoyl_l_ala_d_gamma_glu_l_lys_d_ala_d_ala,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,[Protein]_S_[1_5_O_phosphono_beta_D_ribofurano...,[Protein]_S_[1_5_O_phosphono_beta_D_ribofurano...,,0,[c],[protein]_s_[1_5_o_phosphono_beta_d_ribofurano...,[protein]_s_[1_5_o_phosphono_beta_d_ribofurano...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
mdf5['id'].isna().sum()

66

In [29]:
map_list = pd.concat([mdf,mdf2, mdf3, mdf4])[['mid', 'id', 'BiggID', 'compartment']].dropna(subset='id')# .drop_duplicates()

In [30]:
map_list2 = map_list.sort_values('BiggID').drop_duplicates('mid', keep='first')

In [31]:
map_list2.isna().sum()

mid              0
id               0
BiggID         104
compartment      0
dtype: int64

In [32]:
map_list2['newid'] = map_list2.BiggID + map_list2.compartment
map_list2.newid.value_counts()

alac__S[c]    2
malcoa[c]     2
acald[c]      2
f6p[c]        2
thfglu[c]     2
             ..
cytd[c]       1
dad_2[c]      1
dad_5[c]      1
dadp[c]       1
zn2[e]        1
Name: newid, Length: 621, dtype: int64

In [33]:
# for some metabolites there is more than onemetabolite that links to the same biggid 
map_list2.loc[map_list2.newid.duplicated(keep=False) & ~map_list2.newid.isna()]

,mid,id,BiggID,compartment,newid
387,R_2_3_Dihydroxy_3_methylbutanoate[c],cpd19030,23dhmb,[c],23dhmb[c]
339,2_3_Dihydroxy_3_methylbutanoate[c],cpd02498,23dhmb,[c],23dhmb[c]
69,Acetaldehyde[c],cpd00071,acald,[c],acald[c]
651,Aldehyde[c],cpd00071,acald,[c],acald[c]
492,S_2_Acetolactate[c],cpd19047,alac__S,[c],alac__S[c]
213,2_Acetolactate[c],cpd00668,alac__S,[c],alac__S[c]
70,D_Fructose_6_phosphate[c],cpd00072,f6p,[c],f6p[c]
716,beta_D_Fructose_6_phosphate[c],cpd19035,f6p,[c],f6p[c]
68,Malonyl[c],cpd00070,malcoa,[c],malcoa[c]
740,Malonyl_CoA[c],cpd00070,malcoa,[c],malcoa[c]


In [34]:
map_list2 = map_list2.sort_values('mid').drop_duplicates('newid', keep='first')

In [35]:
df_m2 =pd.merge(df_m, map_list, on='mid', how='left')
df_m2 =pd.merge(df_m2, ref_df, on='id', how='left',suffixes=['','_ref'])
df_m2.head()

,mid,name,formula,charge,compartment_x,cleaned_name,cleaned_mid,id,BiggID,compartment_y,...,pka,pkb,abstract_compound,comprised_of,aliases,smiles,notes,BiggID_ref,cleaned_name_ref,alias
0,amino_acid_pool[c],Amino Acid Pool biomass precursor,,0,[c],amino_acid_pool_biomass_precursor,amino_acid_pool,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amino_sugars[c],Amino sugars pool,,0,[c],amino_sugars_pool,amino_sugars,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Artificial_protein[c],Artificial Protein biomass precursor,,0,[c],artificial_protein_biomass_precursor,artificial_protein,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,biomass[c],Biomass,,0,[c],biomass,biomass,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H2O[c],H2O,H2O,0,[c],h2o,h2o,cpd00001,h2o,[c],...,1:1:15.70,1:1:-1.80,NaN,NaN,Name: H20; H2O; H3O+; HO-; Hydroxide ion; OH; ...,O,GC|EQ|EQU,h2o,h2o,"[Name, H20, H2O, H3O+, HO-, Hydroxide ion,..."


In [36]:
df_m2.loc[df_m2.formula.isin(['']) & ~df_m2.formula_ref.isna(), ['mid', 'formula_ref']]

,mid,formula_ref
53,Zn2[c],Zn
88,Calcium_cation[c],Ca
158,Reduced_ferredoxin[c],Fe2R4S6
159,Oxidized_ferredoxin[c],Fe2R4S6
469,Sodium_cation[c],Na
...,...,...
1135,Hydroquinone[c],C6H6O2
1136,L_Histidinal[c],C6H10N3O
1137,3_D_Glucosyl_1_2_diacylglycerol[c],C11H16O10R2
1138,Cobyrinate[c],C45H52CoN4O14


In [37]:
# update formula for metabolites without formula
for idx, row in  df_m2.loc[df_m2.formula.isin(['']) & ~df_m2.formula_ref.isna(), ['mid', 'formula_ref']].iterrows():
    #print(row['mid'], row['formula_ref'])
    model.metabolites.get_by_id(row['mid']).formula = row['formula_ref']


In [38]:
model.repair()

In [39]:
model.metabolites.get_by_id('Guanine[e]').elements

{'C': 5, 'H': 5, 'N': 5, 'O': 1}

In [40]:
model_dpath = os.path.join('..', 'Model_files')
new_model_fname = 'iSO595v7_with_formula.xml'
new_model_fpath = os.path.join(model_dpath, new_model_fname)

In [41]:
cobra.io.write_sbml_model(model, new_model_fpath)

In [42]:
df_m2.to_excel('MED4_to_Bigg.xlsx')

In [43]:
df_m2.loc[df_m2.name_ref.isna() & ~df_m2.name.isna()& df_m2.name.str.contains('[e]', regex=False)]

,mid,name,formula,charge,compartment_x,cleaned_name,cleaned_mid,id,BiggID,compartment_y,...,pka,pkb,abstract_compound,comprised_of,aliases,smiles,notes,BiggID_ref,cleaned_name_ref,alias


In [45]:
map_list2.loc[map_list2.BiggID.isna() & ~map_list2.id.isna()#& df_m.name.str.contains('[e]', regex=False)
        ]

,mid,id,BiggID,compartment,newid
527,15_16_Dihydrobiliverdin[c],cpd08441,NaN,[c],NaN


In [46]:
model.metabolites

[<Metabolite amino_acid_pool[c] at 0x7ff8987cba30>,
 <Metabolite Amino_sugars[c] at 0x7ff8987cbb50>,
 <Metabolite Artificial_protein[c] at 0x7ff8987cbc70>,
 <Metabolite biomass[c] at 0x7ff8987cbb80>,
 <Metabolite H2O[c] at 0x7ff8987cba60>,
 <Metabolite ATP[c] at 0x7ff8987cbca0>,
 <Metabolite NAD[c] at 0x7ff8987cbcd0>,
 <Metabolite NADH[c] at 0x7ff8987cbd00>,
 <Metabolite NADPH[c] at 0x7ff8987cbd30>,
 <Metabolite NADP[c] at 0x7ff8987cbd60>,
 <Metabolite Oxygen[c] at 0x7ff8987cbd90>,
 <Metabolite ADP[c] at 0x7ff8987cbdc0>,
 <Metabolite Orthophosphate[c] at 0x7ff8987cbdf0>,
 <Metabolite CoA[c] at 0x7ff8987cbe20>,
 <Metabolite CO2[c] at 0x7ff8987cbe50>,
 <Metabolite Diphosphate[c] at 0x7ff8987cbe80>,
 <Metabolite Ammonia[c] at 0x7ff8987cbeb0>,
 <Metabolite UDP[c] at 0x7ff8987cbee0>,
 <Metabolite FAD[c] at 0x7ff8987cbf10>,
 <Metabolite S_Adenosyl_L_methionine[c] at 0x7ff8987cbf40>,
 <Metabolite AMP[c] at 0x7ff8987cbf70>,
 <Metabolite S_Adenosyl_L_homocysteine[c] at 0x7ff8987cbfa0>,
 <Metabo

In [47]:
with model:
    for idx, row in  map_list2.loc[~map_list2.BiggID.isna()].iterrows():
        print(row['mid'], row['newid'])
        
        model.metabolites.get_by_id(row['mid']).id = row['newid']
        model.repair()


10_Formyltetrahydrofolate[c] 10fthf[c]
131_Hydroxy_magnesium_protoporphyrin_IX_13_monomethyl_ester[c] hmppp9[c]
131_Oxo_magnesium_protoporphyrin_IX_13_monomethyl_ester[c] omppp9[c]
1R_6R_6_Hydroxy_2_succinylcyclohexa_2_4_diene_1_carboxylate[c] 2shchc[c]
1_2_Carboxyphenylamino_1_deoxy_D_ribulose_5_phosphate[c] 2cpr5p[c]
1_2_Diacyl_sn_glycerol[c] 12dgr_EC[c]
1_4_Dihydroxy_2_naphthoate[c] dhna[c]
1_5_Phospho_D_ribosyl[c] prbatp[c]
1_5_Phospho_D_ribosyl_5_amino_4_imidazolecarboxylate[c] 5aizc[c]
1_5_Phosphoribosyl_5_amino_4_N_succinocarboxamide_imidazole[c] 25aics[c]
1_5_Phosphoribosyl_5_amino_4_imidazolecarboxamide[c] aicar[c]
1_5_Phosphoribosyl_5_formamido_4_imidazolecarboxamide[c] fprica[c]
1_Acyl_sn_glycerol_3_phosphate[c] 1ag3p_SC[c]
1_Deoxy_D_xylulose_5_phosphate[c] dxyl5p[c]
1_Hydroxy_2_methyl_2_butenyl_4_diphosphate[c] h2mb4p[c]
2R_3S_3_Isopropylmalate[c] 3c2hmp[c]
2S_2_Isopropyl_3_oxosuccinate[c] 3c4mop[c]
2S_4S_4_Hydroxy_2_3_4_5_tetrahydrodipicolinate[c] 23dhdp[c]
2_3_4_5_Tetrahy

GDP_mannose[c] gdpmann[c]
GMP[c] gmp[c]
GTP[c] gtp[c]
Geranyl_diphosphate[c] grdp[c]
Geranylgeranyl_diphosphate[c] ggdp[c]
Glutathione[c] gthrd[c]
Glutathione[e] gthrd[e]
Glutathione_disulfide[c] gthox[c]
Glycerol[c] glyc[c]
Glycerone[c] dha[c]
Glycerone_phosphate[c] dhap[c]
Glycine[c] gly[c]
Glycine[e] gly[e]
Glycogen[c] glycogen[c]
Glycogen[e] glycogen[e]
Glycolaldehyde[c] gcald[c]
Glycolaldehyde[e] gcald[e]
Glycolate[c] glyclt[c]
Glycolate[e] glyclt[e]
Glycyl_tRNAGly[c] glytrna[c]
Glyoxylate[c] glx[c]
Guanine[c] gua[c]
Guanine[e] gua[e]
Guanosine[c] gsn[c]
Guanosine[e] gsn[e]
Guanosine_3_5_bisdiphosphate[c] ppgpp[c]
Guanosine_3_diphosphate_5_triphosphate[c] gdptp[c]
H2O[c] h2o[c]
H2O[car] h2o[car]
H2O[e] h2o[e]
HCO3[e] h2co3[e]
HCO3_[c] h2co3[c]
HCO3_[car] h2co3[car]
H[c] h[c]
H[car] h[car]
H[e] h[e]
H[p] h[p]
H[th] h[th]
Heme[c] pheme[c]
Heme_O[c] hemeO[c]
Hexadecanoic_acid[c] hdca[c]
Hexadecanoyl[c] palmACP[c]
Hexanoyl[c] hexACP[c]
Holo_[carboxylase][c] btnCCP[c]
Hydrogen_peroxide

Thiamin_diphosphate[e] thmpp[e]
Thiamin_monophosphate[c] thmmp[c]
Thiamine[c] thm[c]
Thioredoxin[c] trdrd[c]
Thioredoxin[e] trdrd[e]
Thioredoxin_disulfide[c] trdox[c]
Thiosulfate[c] tsul[c]
Thiosulfate[e] tsul[e]
Thymidine[c] thymd[c]
Thymidine[e] thymd[e]
Triphosphate[c] pppi[c]
UDP[c] udp[c]
UDP_2_3_bis3_hydroxytetradecanoylglucosamine[c] u23ga[c]
UDP_3_O_3_hydroxytetradecanoyl_D_glucosamine[c] u3hga[c]
UDP_3_O_3_hydroxytetradecanoyl_N_acetylglucosamine[c] u3aga[c]
UDP_6_sulfoquinovose[c] udpsq[c]
UDP_D_galacturonate[c] udpgalur[c]
UDP_N_acetyl_3_1_carboxyvinyl_D_glucosamine[c] uaccg[c]
UDP_N_acetyl_alpha_D_glucosamine[c] uacgam[c]
UDP_N_acetylmuramate[c] uamr[c]
UDP_N_acetylmuramoyl_L_alanine[c] uama[c]
UDP_N_acetylmuramoyl_L_alanyl_D_gamma_glutamyl_meso_2_6_diaminopimelate[c] uggmd[c]
UDP_N_acetylmuramoyl_L_alanyl_D_glutamate[c] uamag[c]
UDP_N_acetylmuramoyl_L_alanyl_D_glutamyl_6_carboxy_L_lysyl_D_alanyl_D_alanine[c] uggmda[c]
UDP_glucose[c] udpg[c]
UDP_glucuronate[c] udpglcur[c]
U

In [48]:
model.metabolites

[<Metabolite amino_acid_pool[c] at 0x7ff8987cba30>,
 <Metabolite Amino_sugars[c] at 0x7ff8987cbb50>,
 <Metabolite Artificial_protein[c] at 0x7ff8987cbc70>,
 <Metabolite biomass[c] at 0x7ff8987cbb80>,
 <Metabolite h2o[c] at 0x7ff8987cba60>,
 <Metabolite atp[c] at 0x7ff8987cbca0>,
 <Metabolite nad[c] at 0x7ff8987cbcd0>,
 <Metabolite nadh[c] at 0x7ff8987cbd00>,
 <Metabolite nadph[c] at 0x7ff8987cbd30>,
 <Metabolite nadp[c] at 0x7ff8987cbd60>,
 <Metabolite o2[c] at 0x7ff8987cbd90>,
 <Metabolite adp[c] at 0x7ff8987cbdc0>,
 <Metabolite pi[c] at 0x7ff8987cbdf0>,
 <Metabolite coa[c] at 0x7ff8987cbe20>,
 <Metabolite co2[c] at 0x7ff8987cbe50>,
 <Metabolite ppi[c] at 0x7ff8987cbe80>,
 <Metabolite nh3[c] at 0x7ff8987cbeb0>,
 <Metabolite udp[c] at 0x7ff8987cbee0>,
 <Metabolite fad[c] at 0x7ff8987cbf10>,
 <Metabolite amet[c] at 0x7ff8987cbf40>,
 <Metabolite amp[c] at 0x7ff8987cbf70>,
 <Metabolite ahcys[c] at 0x7ff8987cbfa0>,
 <Metabolite pyr[c] at 0x7ff8987cbfd0>,
 <Metabolite glu__L[c] at 0x7ff8987

In [ ]:
mapdf[['mid','name','formula', 'formula_ref',]]

In [49]:
model_dpath = os.path.join('..', 'Model_files')
new_model_fname = 'iSO595v7_BiGG.xml'
new_model_fpath = os.path.join(model_dpath, new_model_fname)

In [50]:
cobra.io.write_sbml_model(model, new_model_fpath)